# Environment

In [1]:
import gc
import os
import importlib
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import celloracle as co
co.__version__

sys.path.insert(0, "/home/michal.kubacki/Githubs/Re-MEND/code/External_Datasets/GeneSet_Derivation/Herring_celloracle/helpers")

import config
importlib.reload(config)
from config import *
n_cpus = 32


which: no R in (/home/michal.kubacki/Downloads/usr/share/code/bin:/home/michal.kubacki/.local/bin:/home/michal.kubacki/micromamba/condabin:/home/michal.kubacki/cuda/bin:/home/michal.kubacki/.local/bin:/cm/shared/apps/slurm/current/sbin:/cm/shared/apps/slurm/current/bin:/cm/local/apps/environment-modules/4.5.3//bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/usr/sbin:/cm/local/apps/environment-modules/4.5.3/bin:/opt/dell/srvadmin/bin:.:/home/michal.kubacki/lib/hdf5-1.10.5/bin:/scratch/michal.kubacki/modules/bedtools2/bin)


In [2]:
reference = "hg19"

neurons_set = "all_excitatory_all_ages"
# neurons_set = "all_excitatory"

cells_dict = {
    "all_excitatory"            :   ['L5-6_TLE4', 'L2-3_CUX2', 'L4_RORB', 'L5-6_THEMIS', 'PN_dev'],
    "all_excitatory_all_ages"   :   ['L5-6_TLE4', 'L2-3_CUX2', 'L4_RORB', 'L5-6_THEMIS', 'PN_dev']
}

ages_dict = {
    "all_excitatory"            :   ['1m','3m','6m','10m','1y','2y','4y','ga22','ga24'],
    "all_excitatory_all_ages"   :   ['1m','3m','6m','10m','1y','2y','4y','6y','10y','16y','20y','40y','ga22','ga24']
}

output_dir, input_dir, root_dir, tmp_dir, in_dir_from_scenic = set_custom_folders(reference, neurons_set)

sel_celltypes  = cells_dict[neurons_set]
sel_ages = ages_dict[neurons_set]

root_dir: /group/testa/michal.kubacki/herring
out_dir: /group/testa/michal.kubacki/herring/output_hg19_all_excitatory/celloracle
in_dir: /group/testa/michal.kubacki/herring/data
tmp_dir: /group/testa/michal.kubacki/herring/celloracle/tmp


In [3]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

# Check results

In [7]:
cell_type = sel_celltypes[0]

In [8]:
file_path = os.path.join(output_dir, f"{cell_type}.celloracle.parquet")

In [9]:
df = pd.read_parquet(file_path)

In [10]:
df.shape

(21874, 1011)

In [11]:
# total number of non zero elements
result = df.iloc[:, 2:].astype(bool).sum().sum()
print(result)

1965036


In [12]:
selected_columns = df.iloc[:, 2:]
row_counts = selected_columns.apply(lambda row: len(row[row != 0.0]), axis=1)
print(list(row_counts))

first_row = df.iloc[0]
non_zero_elements = first_row[first_row != 0]
print(non_zero_elements)

[123, 114, 131, 99, 75, 76, 86, 93, 73, 114, 114, 109, 98, 53, 105, 80, 90, 71, 76, 74, 42, 104, 81, 58, 58, 80, 41, 150, 58, 109, 109, 124, 97, 95, 120, 74, 75, 75, 87, 36, 67, 51, 105, 105, 87, 100, 122, 62, 83, 78, 87, 89, 99, 70, 74, 101, 105, 104, 67, 63, 124, 68, 116, 70, 75, 78, 118, 105, 141, 123, 46, 64, 90, 60, 102, 72, 72, 87, 64, 85, 92, 44, 116, 113, 93, 101, 71, 94, 132, 49, 57, 81, 47, 74, 110, 80, 80, 88, 125, 117, 48, 92, 84, 101, 73, 95, 96, 88, 73, 67, 75, 93, 93, 97, 96, 54, 36, 70, 79, 119, 69, 114, 126, 126, 70, 56, 90, 80, 84, 93, 93, 88, 86, 112, 112, 42, 70, 126, 58, 110, 86, 132, 76, 88, 63, 106, 75, 94, 56, 88, 107, 128, 76, 70, 136, 73, 81, 81, 50, 49, 93, 121, 121, 100, 29, 63, 100, 84, 93, 73, 158, 112, 57, 51, 73, 84, 105, 105, 150, 57, 122, 96, 67, 67, 86, 108, 127, 87, 94, 36, 121, 78, 87, 143, 82, 68, 145, 58, 138, 95, 113, 119, 85, 97, 62, 109, 95, 117, 83, 64, 90, 119, 114, 92, 97, 42, 98, 103, 71, 57, 57, 71, 71, 68, 79, 104, 75, 86, 146, 103, 59, 1

## Compare with `2023_11_tfi.celloracle.parquet`

In [13]:
base_GRN = pd.read_parquet(os.path.join(input_dir, "2023_11_tfi.celloracle.parquet"), engine='pyarrow')

In [14]:
base_GRN.head()

,peak_id,gene_short_name,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,AHR,AHRR,AIRE,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100009478_100010332,DNMBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr10_100045768_100046671,CHUK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100045768_100046671,ERLIN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100185577_100186445,ERLIN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_100229128_100229954,CHUK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
base_GRN.shape

(272543, 1084)

In [16]:
selected_columns = base_GRN.iloc[:, 2:]
row_counts = selected_columns.apply(lambda row: len(row[row != 0.0]), axis=1)
print(list(row_counts[:100]))

first_row = base_GRN.iloc[0]
non_zero_elements = first_row[first_row != 0]
print(non_zero_elements)

[104, 47, 47, 122, 117, 85, 78, 35, 35, 35, 35, 35, 98, 98, 98, 98, 98, 98, 44, 44, 114, 68, 86, 85, 61, 61, 61, 73, 77, 58, 131, 117, 117, 104, 104, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 135, 135, 135, 135, 158, 158, 158, 158, 158, 158, 158, 158, 158, 65, 65, 65, 65, 113, 113, 113, 113, 113, 122, 122, 122, 122, 122, 122, 122, 122, 122, 125, 57, 63, 63, 63, 63, 61, 42, 42, 42, 92, 76, 76, 76, 76, 76, 76, 76, 76, 76, 110, 110, 110, 110]
peak_id            chr10_100009478_100010332
gene_short_name                        DNMBP
ATF3                                     1.0
BACH1                                    1.0
BACH2                                    1.0
                             ...            
ZIC3                                     1.0
ZIC4                                     1.0
ZIC5                                     1.0
ZNF143                                   1.0
ZNF350                                   1.0
Name: 0, Length: 106, dtype: object


In [17]:
result2 = base_GRN.iloc[:, 2:].astype(bool).sum().sum()
print(result2)

21064823


In [18]:
print(result/result2)

0.09328518924654625
